### [772\. Basic Calculator III](https://leetcode.com/problems/basic-calculator-iii/)

Difficulty: **Hard**


Implement a basic calculator to evaluate a simple expression string.

The expression string may contain open `(` and closing parentheses `)`, the plus `+` or minus sign `-`, **non-negative** integers and empty spaces .

The expression string contains only non-negative integers, `+`, `-`, `*`, `/` operators , open `(` and closing parentheses `)` and empty spaces . The integer division should truncate toward zero.

You may assume that the given expression is always valid. All intermediate results will be in the range of `[-2147483648, 2147483647]`.

Some examples:

```
"1 + 1" = 2
" 6-4 / 2 " = 4
"2*(5+5*2)/3+(6/2+8)" = 21
"(2+6* 3+5- (3*14/7+2)*5)+3"=-12
```

**Note:** **Do not** use the `eval` built-in library function.

In [34]:
# Reverse Polish Notation (RPN)
# need to special handle unary operator (-), e.g. 1-(-7)
from typing import List

class Solution:
    def calculate(self, s: str) -> int:
        # shunting-yard algorithm
        def infix2postfix(exp: str) -> List[str]:
            PRECEDENCE = {'(': 0, '+': 1, '-': 1, '*': 2, '/': 2, ')': 3}
            stack, q, digit_cache = [], [], []
            prev_isdigit = False
            for ch in exp:
                if ch == ' ': # skip whitespaces
                    continue
                elif ch not in '()+-*/': # add integer into queue
                    digit_cache.append(ch)
                    prev_isdigit = True
                else:
                    # if ch is a '+' or '-' sign before a number, the prev valid ch should be either a number or a ')'
                    if not prev_isdigit and ch in '+-':
                        digit_cache.append(ch)
                        continue
                    _addDigit(digit_cache, q)
                    if ch == ')':
                        while stack and stack[-1] != '(':
                            q.append(stack.pop())
                        stack.pop() # remove '('
                        prev_isdigit = True
                        continue
                    elif ch in '+-*/':
                        while stack and PRECEDENCE[stack[-1]] >= PRECEDENCE[ch]:
                            q.append(stack.pop())
                        stack.append(ch)
                    elif ch == '(':
                        stack.append(ch)
                    prev_isdigit = False
                        
            _addDigit(digit_cache, q)
            # important: pop all remaining operators from stack and add to the queue
            while stack:
                q.append(stack.pop())
            return q
        
        def _addDigit(digit_cache: List[str], queue: List[str]):
            if digit_cache:
                queue.append(''.join(digit_cache))
                digit_cache.clear()
            
        def evalPostfix(exp: List[str]) -> int:
            stack = []
            for token in exp:
                if token not in '+-*/': # integer
                    stack.append(int(token))
                else:
                    n2, n1 = stack.pop(), stack.pop()# if stack else None
                    #n3 = _sign(n2, token) if n1 is None else _calc(n1, n2, token)
                    n3 = _calc(n1, n2, token)
                    stack.append(n3)
            return stack[-1]
        
        def _sign(n: int, sign: str) -> int:
            if sign == '+':
                return n
            elif sign == '-':
                return -n
            
        def _calc(n1: int, n2: int, op: str) -> int:
            if op == '+':
                return n1 + n2
            elif op == '-':
                return n1 - n2
            elif op == '*':
                return n1 * n2
            elif op == '/':
                return n1 // n2
            else:
                raise Exception("Operator isn't supported {}".format(op))
        
        postfix = infix2postfix(s)
        print("postfix: {}".format(postfix))
        return evalPostfix(postfix)

In [35]:
Solution().calculate("(2+(4/5+2)-3)*(6+8)")

postfix: ['2', '4', '5', '/', '2', '+', '+', '3', '-', '6', '8', '+', '*']


14

In [36]:
Solution().calculate("10 / (40+50)")

postfix: ['10', '40', '50', '+', '/']


0

In [37]:
Solution().calculate("-1+4*3/3/3")

postfix: ['-1', '4', '3', '*', '3', '/', '3', '/', '+']


0

In [38]:
Solution().calculate("-1+(-4+3/3/3)")

postfix: ['-1', '-4', '3', '3', '/', '3', '/', '+', '+']


-5

In [39]:
Solution().calculate("1-(-7)")

postfix: ['1', '-7', '-']


8

In [40]:
Solution().calculate("(2+6* 3+5- (3*14/7+2)*5)+3")

postfix: ['2', '6', '3', '*', '+', '5', '+', '3', '14', '*', '7', '/', '2', '+', '5', '*', '-', '3', '+']


-12